In [ ]:
# prompt: mount google drive

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import requests
import os
import cv2
import numpy as np

# Images from Flickr

In [ ]:
# def save_images_from_flickr(api_key, query, num_images, dataset_directory):
#     if not os.path.exists(dataset_directory):
#         os.makedirs(dataset_directory)
#         print(f"Created directory: {dataset_directory}")

#     url = "https://api.flickr.com/services/rest/"
#     params = {
#         "method": "flickr.photos.search",
#         "api_key": api_key,
#         "text": query,
#         "format": "json",
#         "nojsoncallback": 1,
#         "per_page": num_images
#     }

#     response = requests.get(url, params=params)
#     response.raise_for_status()

#     photos = response.json()["photos"]["photo"]
#     print("Photos found:", len(photos))

#     for i, photo in enumerate(photos):
#         photo_url = f"https://live.staticflickr.com/{photo['server']}/{photo['id']}_ {photo['secret']}.jpg"
#         print("Downloading:", photo_url)

#         img_response = requests.get(photo_url)
#         img_response.raise_for_status()

#         with open(os.path.join(dataset_directory, f"image_{i}.jpg"), "wb") as file:
#             file.write(img_response.content)
#             print(f"Saved image_{i}.jpg")


# queries = ["healthy", "unhealthy", "glamorous", "drab",
#            "rugged", "gentle", "fun", "dull"]

# api_key = "6c5f4e8afdb73883214c9b9471629b81"

# num_images = 200

# for query in queries:
#     dataset_directory = "/content/" + query
#     save_images_from_flickr(api_key, query, num_images, dataset_directory)

## Preprocess

In [ ]:
def preprocess_image(img_path, target_size=(224, 224)):

    img = cv2.imread(img_path)
    img = cv2.resize(img, target_size)
    img = img.astype('float32') / 255.0
    return img

# def preprocess_images_from_folders(folders, target_size=(224, 224)):
#     preprocessed_images = {}

#     for folder in folders:
#         label = os.path.basename(folder)
#         preprocessed_images[label] = []

#         for img_file in os.listdir(folder):
#             img_path = os.path.join(folder, img_file)
#             if os.path.isfile(img_path):
#                 preprocessed_images[label].append(preprocess_image(img_path, target_size))

#     return preprocessed_images

# folders = ['/content/healthy', '/content/unhealthy', '/content/glamorous',
#            '/content/drab', '/content/rugged', '/content/gentle',
#            '/content/fun', '/content/dull']

# preprocessed_images = preprocess_images_from_folders(folders)

# for label, images in preprocessed_images.items():
#     print(f"Label: {label}, Number of Images: {len(images)}")

## Save to google drive

In [ ]:
# def save_preprocessed_images(preprocessed_images, target_directory):
#     if not os.path.exists(target_directory):
#         os.makedirs(target_directory)

#     for label, images in preprocessed_images.items():
#         label_dir = os.path.join(target_directory, label)
#         if not os.path.exists(label_dir):
#             os.makedirs(label_dir)

#         for i, img in enumerate(images):
#             img_path = os.path.join(label_dir, f"{label}_{i}.jpg")
#             cv2.imwrite(img_path, img * 255)  # Rescale back to 0-255 range

# # Shared Google Drive folder path
# shared_folder_path = '/content/drive/My Drive/ConsumerCompass/data'  # Update with the actual path

# # Save images to the shared folder
# save_preprocessed_images(preprocessed_images, shared_folder_path)

## Splitting for training model

In [ ]:
import numpy as np
import os
from sklearn.model_selection import train_test_split
from google.colab import drive

# Google Drive folder path
base_folder = '/content/drive/My Drive/ConsumerCompass/data'

# Update folder paths
folders = [os.path.join(base_folder, folder_name) for folder_name in ['healthy', 'unhealthy', 'glamorous', 'drab', 'rugged', 'gentle', 'fun', 'dull']]

# Rest of your code for processing
label_map = {'healthy': 0, 'unhealthy': 1, 'glamorous': 2, 'drab': 3, 'rugged': 4, 'gentle': 5, 'fun': 6, 'dull': 7}
all_images = []
all_labels = []

for folder in folders:
    label_index = label_map[os.path.basename(folder)]
    print(folder)
    for img_file in os.listdir(folder):
        img_path = os.path.join(folder, img_file)
        if os.path.isfile(img_path):
            img = preprocess_image(img_path)
            label_vector = [0] * 8
            label_vector[label_index] = 1
            all_images.append(img)
            all_labels.append(label_vector)

all_images = np.array(all_images)
all_labels = np.array(all_labels)

train_images, test_images, train_labels, test_labels = train_test_split(all_images, all_labels, test_size=0.3)
val_images, test_images, val_labels, test_labels = train_test_split(test_images, test_labels, test_size=0.5)

/content/drive/My Drive/ConsumerCompass/data/healthy
/content/drive/My Drive/ConsumerCompass/data/unhealthy
/content/drive/My Drive/ConsumerCompass/data/glamorous
/content/drive/My Drive/ConsumerCompass/data/drab
/content/drive/My Drive/ConsumerCompass/data/rugged
/content/drive/My Drive/ConsumerCompass/data/gentle
/content/drive/My Drive/ConsumerCompass/data/fun
/content/drive/My Drive/ConsumerCompass/data/dull


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

label_map = {'healthy': 0, 'unhealthy': 1, 'glamorous': 2, 'drab': 3,
             'rugged': 4, 'gentle': 5, 'fun': 6, 'dull': 7}
all_images = []
all_labels = []

for folder in folders:
    label_index = label_map[os.path.basename(folder)]
    for img_file in os.listdir(folder):
        img_path = os.path.join(folder, img_file)
        if os.path.isfile(img_path):
            img = preprocess_image(img_path)
            label_vector = [0] * 8
            label_vector[label_index] = 1
            all_images.append(img)
            all_labels.append(label_vector)

all_images = np.array(all_images)
all_labels = np.array(all_labels)

train_images, test_images, train_labels, test_labels = train_test_split(all_images, all_labels, test_size=0.3)
val_images, test_images, val_labels, test_labels = train_test_split(test_images, test_labels, test_size=0.5)

In [ ]:
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D
# from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.layers import Dropout, BatchNormalization

# # model = Sequential([
# #     Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
# #     MaxPooling2D(pool_size=(2, 2)),
# #     Flatten(),
# #     Dense(128, activation='relu'),
# #     Dense(8, activation='sigmoid')
# # ])

# model = Sequential([
#     Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
#     MaxPooling2D(pool_size=(2, 2)),
#     BatchNormalization(),

#     Conv2D(64, (3, 3), activation='relu'),
#     MaxPooling2D(pool_size=(2, 2)),
#     BatchNormalization(),

#     Conv2D(128, (3, 3), activation='relu'),
#     MaxPooling2D(pool_size=(2, 2)),
#     BatchNormalization(),

#     Flatten(),
#     Dense(256, activation='relu'),
#     Dropout(0.5),
#     Dense(128, activation='relu'),
#     Dropout(0.5),
#     Dense(8, activation='sigmoid')
# ])

# model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])
# model.fit(train_images, train_labels, validation_data=(val_images, val_labels), epochs=10)

# test_loss, test_accuracy = model.evaluate(test_images, test_labels)
# model.save('/content/model2')

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

model = Sequential([
    # # Conv0
    # Conv2D(filters=64, kernel_size=(15, 15), strides=(4, 4), activation='relu', input_shape=(224, 224, 3)),
    # # MaxPool1
    # MaxPooling2D(pool_size=(3, 3), strides=(2, 2)),
    # # Norm1
    # BatchNormalization(),

    # Conv1
    Conv2D(filters=96, kernel_size=(11, 11), strides=(4, 4), activation='relu', input_shape=(224, 224, 3)),
    # MaxPool1
    MaxPooling2D(pool_size=(3, 3), strides=(2, 2)),
    # Norm1
    BatchNormalization(),

    # Conv2
    Conv2D(filters=256, kernel_size=(5, 5), padding='same', activation='relu'),
    # MaxPool2
    MaxPooling2D(pool_size=(3, 3), strides=(2, 2)),
    # Norm2
    BatchNormalization(),

    # Conv3
    Conv2D(filters=384, kernel_size=(3, 3), padding='same', activation='relu'),

    # Conv4
    Conv2D(filters=384, kernel_size=(3, 3), padding='same', activation='relu'),

    # Conv5
    Conv2D(filters=256, kernel_size=(3, 3), padding='same', activation='relu'),
    # MaxPool5
    MaxPooling2D(pool_size=(3, 3), strides=(2, 2)),

    Flatten(),

    # FC6
    Dense(4096, activation='relu'),
    # Dropout FC6
    Dropout(0.5),

    # FC7
    Dense(4096, activation='relu'),
    # Dropout FC7
    Dropout(0.5),

    # FC8
    Dense(8, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

history = model.fit(
    train_images,
    train_labels,
    validation_data=(val_images, val_labels),
    epochs=10
)

test_loss, test_accuracy = model.evaluate(test_images, test_labels)

model.save('/content/model_multilabel_brandimagenet')

In [ ]:
# import tensorflow as tf
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

# model = Sequential([
#     Conv2D(filters=96, kernel_size=(11, 11), strides=(4, 4), activation='relu', input_shape=(224, 224, 3)),
#     MaxPooling2D(pool_size=(3, 3), strides=(2, 2)),
#     BatchNormalization(),

#     Conv2D(filters=256, kernel_size=(5, 5), padding='same', activation='relu'),
#     MaxPooling2D(pool_size=(3, 3), strides=(2, 2)),
#     BatchNormalization(),

#     Conv2D(filters=384, kernel_size=(3, 3), padding='same', activation='relu'),
#     Conv2D(filters=384, kernel_size=(3, 3), padding='same', activation='relu'),
#     Conv2D(filters=384, kernel_size=(3, 3), padding='same', activation='relu'),
#     Conv2D(filters=256, kernel_size=(3, 3), padding='same', activation='relu'),
#     MaxPooling2D(pool_size=(3, 3), strides=(2, 2)),

#     Flatten(),

#     Dense(4096, activation='relu'),
#     Dropout(0.5),
#     Dense(4096, activation='relu'),
#     Dropout(0.5),
#     Dense(8, activation='sigmoid')
# ])

# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# model.summary()

# history = model.fit(
#     train_images,
#     train_labels,
#     validation_data=(val_images, val_labels),
#     epochs=15
# )

# test_loss, test_accuracy = model.evaluate(test_images, test_labels)
# model.save('/content/model_multilabel_brandimagenet')


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 54, 54, 96)        34944     
                                                                 
 max_pooling2d_6 (MaxPoolin  (None, 26, 26, 96)        0         
 g2D)                                                            
                                                                 
 batch_normalization_4 (Bat  (None, 26, 26, 96)        384       
 chNormalization)                                                
                                                                 
 conv2d_13 (Conv2D)          (None, 26, 26, 256)       614656    
                                                                 
 max_pooling2d_7 (MaxPoolin  (None, 12, 12, 256)       0         
 g2D)                                                            
                                                      

KeyboardInterrupt: 

In [ ]:
# import tensorflow as tf
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

# model = Sequential([
#     # # Conv0
#     # Conv2D(filters=64, kernel_size=(15, 15), strides=(4, 4), activation='relu', input_shape=(224, 224, 3)),
#     # # MaxPool1
#     # MaxPooling2D(pool_size=(3, 3), strides=(2, 2)),
#     # # Norm1
#     # BatchNormalization(),

#     # Conv1
#     Conv2D(filters=96, kernel_size=(11, 11), strides=(4, 4), activation='relu', input_shape=(224, 224, 3)),
#     # MaxPool1
#     MaxPooling2D(pool_size=(3, 3), strides=(2, 2)),
#     # Norm1
#     BatchNormalization(),

#     # Conv2
#     Conv2D(filters=256, kernel_size=(5, 5), padding='same', activation='relu'),
#     # MaxPool2
#     MaxPooling2D(pool_size=(3, 3), strides=(2, 2)),
#     # Norm2
#     BatchNormalization(),

#     # Conv3
#     Conv2D(filters=384, kernel_size=(3, 3), padding='same', activation='relu'),

#     # Conv4
#     Conv2D(filters=384, kernel_size=(3, 3), padding='same', activation='relu'),

#     # Conv4.2
#     Conv2D(filters=384, kernel_size=(3, 3), padding='same', activation='relu'),

#     # Conv5
#     Conv2D(filters=256, kernel_size=(3, 3), padding='same', activation='relu'),
#     # MaxPool5
#     MaxPooling2D(pool_size=(3, 3), strides=(2, 2)),

#     Flatten(),

#     # FC6
#     Dense(4096, activation='relu'),
#     # Dropout FC6
#     Dropout(0.5),

#     # FC7
#     Dense(4096, activation='relu'),
#     # Dropout FC7
#     Dropout(0.5),

#     # FC8
#     Dense(8, activation='sigmoid')
# ])

# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# model.summary()

# history = model.fit(
#     train_images,
#     train_labels,
#     validation_data=(val_images, val_labels),
#     epochs=15
# )

# test_loss, test_accuracy = model.evaluate(test_images, test_labels)

# model.save('/content/model_multilabel_brandimagenet')

In [ ]:
import numpy as np
from tensorflow.keras.metrics import BinaryAccuracy

predictions = model.predict(test_images)

accuracy_metrics = []
for i in range(0, predictions.shape[1], 2):
    metric = BinaryAccuracy()
    metric.update_state(test_labels[:, i:i+2], predictions[:, i:i+2])
    accuracy_metrics.append(metric.result().numpy())

class_pairs = ['healthy-unhealthy', 'glamorous-drab', 'rugged-gentle', 'fun-dull']
for i, acc in enumerate(accuracy_metrics):
    print(f"Accuracy for {class_pairs[i]}: {acc}")

# 8/8 [==============================] - 11s 1s/step
# Accuracy for healthy-unhealthy: 0.8640167117118835
# Accuracy for glamorous-drab: 0.9163179993629456
# Accuracy for rugged-gentle: 0.8870292901992798
# Accuracy for fun-dull: 0.8891213536262512

8/8 [==============================] - 10s 1s/step
Accuracy for healthy-unhealthy: 0.8682008385658264
Accuracy for glamorous-drab: 0.8598326444625854
Accuracy for rugged-gentle: 0.8619247078895569
Accuracy for fun-dull: 0.910041868686676


In [ ]:
from tensorflow.keras.models import load_model

model = load_model('/content/model2')

image_path = '/content/test/pic1.jpg' # upload to google collab manually

img = preprocess_image(image_path)

img_batch = np.expand_dims(img, axis=0)

predictions = model.predict(img_batch)

final_classification = []

for i in range(0, len(predictions[0]), 2):
    if predictions[0][i] > predictions[0][i+1]:
        final_classification.extend([1, 0])
    else:
        final_classification.extend([0, 1])

print(final_classification)



In [ ]:
queries = ["healthy", "unhealthy", "glamorous", "drab",
           "rugged", "gentle", "fun", "dull"]

def normalize_and_print_predictions(predictions):
    normalized_predictions = []

    for i in range(0, len(predictions[0]), 2):
        pair_sum = predictions[0][i] + predictions[0][i+1]
        normalized_pair = [predictions[0][i] / pair_sum, predictions[0][i+1] / pair_sum]
        normalized_predictions.extend(normalized_pair)

    # Print the normalized predictions
    for i in range(0, len(normalized_predictions), 2):
        print(f"{queries[i]}: [{normalized_predictions[i]:.2f}, {normalized_predictions[i+1]:.2f}]")

normalize_and_print_predictions(predictions)


In [ ]:
# [0, 1, 0, 1, 1, 0, 0, 1]

In [ ]:
# healthy: [0.52, 0.48]
# glamorous: [0.11, 0.89]
# rugged: [0.34, 0.66]
# fun: [0.58, 0.42]

In [ ]:
# healthy: [0.35, 0.65]
# glamorous: [0.81, 0.19]
# rugged: [0.00, 1.00]
# fun: [0.01, 0.99]

In [ ]:
# 0 at index 0 (healthy): The model predicts that the image is not 'healthy'.
# 1 at index 1 (unhealthy): The model predicts that the image is 'unhealthy'.
# 0 at index 2 (glamorous): The model predicts that the image is not 'glamorous'.
# 0 at index 3 (drab): The model predicts that the image is not 'drab'.
# 1 at index 4 (rugged): The model predicts that the image is 'rugged'.
# 0 at index 5 (gentle): The model predicts that the image is not 'gentle'.
# 1 at index 6 (fun): The model predicts that the image is 'fun'.
# 0 at index 7 (dull): The model predicts that the image is not 'dull'.